In [1]:
import xalpha as xa
import pandas as pd

xa.set_display("notebook")

holdings.py is found and loaded within xalpha dir


<IPython.core.display.Javascript object>

# 指定标的的交易分析

* 想要进行交易分析首先需要交易单

In [2]:
path = "../../tests/demo.csv"

In [3]:
read = xa.record(path)
# record 类用来从指定位置读取 csv 文件，并将交易单数据存储在 self.status 中
# 该类可直接接受 pd.read_csv() 的多样的关键字参数，比如 skiprows=1, 这样就可以在真实的记账单首行标注基金名称，而不影响程序处理

In [4]:
read.status
#  一个示例的交易单演示
## 特别注意由于基金申购是按照金额，赎回是按照份额，因此交易单采用此惯例，正数代表申购金额，负数代表赎回份额
## 数据理论上小数点后两位有效，不过从第二位小数开始，数字起到了一些特殊标记的作用，因此仅完美支持一位小数，对于绝大多数人完全够用
## 金额是不需要考虑申购和赎回费的，简单记录您申购的总金额或赎回的份额，程序将完全充分的考虑所有申购赎回折算分红等情况，确保和真实持仓波动完全一致
## 在原始的 csv 形式中，0 不必要显式写出，直接空过去就好
## 小数点第二位的5，如果当天恰为该基金的登记分红日，则视为选择了分红再投入，否则默认分红拿现金
## 若负数绝对值小于 0.005，则视为赎回的份额比例而非份额数目， －0.005 表示清仓，依此线性缩放
## 更进一步的，非交易交易记录或是赎回份额超过持有总数， 程序都将作出协调自洽的处理
## 账单还可额外增加 property 行，控制金额赎回，分红默认再投入等更多细节，具体请参考文档

,date,001180,003376,100032,001469,164818
0,2016-09-28,200.0,0.0,700.00,0.000,270.0
1,2017-10-17,100.0,300.0,225.00,0.000,0.0
2,2017-12-12,-90.0,242.0,-80.00,380.000,0.0
3,2018-01-18,0.0,0.0,200.05,100.000,0.0
4,2018-07-05,300.0,200.0,770.00,-100.000,210.0
5,2018-07-26,0.0,-10000.0,0.00,-0.005,0.0


* 现在分析一下 001180 基金的交易情形

In [5]:
f = xa.fundinfo("001180")

In [6]:
f

广发医药卫生联接A

In [7]:
f_t = xa.trade(f, read.status)  # 给定标的和账单，交易就生成了

In [8]:
f_t.aim  # 查看该交易的标的

广发医药卫生联接A

In [9]:
f_t.dailyreport()  # 该交易截止昨天的信息总结，也可以给定'2018-01-01'形式的时间参数，来了解任意一天的基金持有概况

,基金名称,基金代码,当日净值,单位成本,持有份额,基金现值,基金总申购,历史最大占用,基金持有成本,基金分红与赎回,换手率,基金收益总额,投资收益率
0,广发医药卫生联接A,001180,0.9977,0.831,630.39,628.94,600.0,523.86,523.86,76.14,0.180222,105.08,20.0588


In [11]:
f_t.xirrrate("2018-07-01")  # 计算截止7.1的 xirr 收益率，最客观的年化收益指标

0.04879311024975706

In [10]:
f_t.v_totvalue()  # 基金持有金额的变动可视化

In [11]:
f_t.v_tradecost()  # 同时比较基金净值和自己交易的单位成本的变动
# 上面橙色点代表买入，蓝色点代表卖出，点的大小代表买卖数量

In [12]:
f_t.v_tradevolume(freq="W")  # 基金交易量的可视化

其他基金的交易分析同理。除了基金，指数类和货币基金类也可以作为交易标的进行交易和分析